# Inference code for YellowKing's model from  DL Sprint 2022
https://www.kaggle.com/code/sameen53/yellowking-dlsprint-inference

In [4]:
!cp -r ../input/python-packages2 ./

In [5]:
!tar xvfz ./python-packages2/jiwer.tgz
!pip install ./jiwer/jiwer-2.3.0-py3-none-any.whl -f ./ --no-index
!tar xvfz ./python-packages2/normalizer.tgz
!pip install ./normalizer/bnunicodenormalizer-0.0.24.tar.gz -f ./ --no-index
!tar xvfz ./python-packages2/pyctcdecode.tgz
!pip install ./pyctcdecode/attrs-22.1.0-py2.py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/exceptiongroup-1.0.0rc9-py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/hypothesis-6.54.4-py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/pygtrie-2.5.0.tar.gz -f ./ --no-index --no-deps
!pip install ./pyctcdecode/sortedcontainers-2.4.0-py2.py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/pyctcdecode-0.4.0-py2.py3-none-any.whl -f ./ --no-index --no-deps

!tar xvfz ./python-packages2/pypikenlm.tgz
!pip install ./pypikenlm/pypi-kenlm-0.1.20220713.tar.gz -f ./ --no-index --no-deps



jiwer/
jiwer/jiwer-2.3.0-py3-none-any.whl
jiwer/python-Levenshtein-0.12.2.tar.gz
jiwer/setuptools-65.3.0-py3-none-any.whl
Looking in links: ./
Processing ./jiwer/jiwer-2.3.0-py3-none-any.whl
jiwer is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
normalizer/
normalizer/bnunicodenormalizer-0.0.24.tar.gz
Looking in links: ./
Processing ./normalizer/bnunicodenormalizer-0.0.24.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for bnunicodenormalizer: filename=bnunicodenormalizer-0.0.24-py3-none-any.whl size=17628 sha256=8fe93fe7595366471fc9bec30dd07c8f47af12520e8521302e6cb6ad9327aaaa
  Stored in directory: /root/.cache/pip/wheels/3f/7a/fd/fde270138c2a0d8efd0e9ce20f760c2e3a0fc4317975237cb8
Successfully built bnunicodenormalizer
  Attempting uninstall: bnunicodenormalizer
    Found existing installation: bnunicodenormalizer 0.0.24
    Uninstalling bnunicodenormalizer-0.0.24:
      Successfully uninstal

In [6]:
!pip install openpyxl

ERROR: Could not find a version that satisfies the requirement openpyxl (from versions: none)
ERROR: No matching distribution found for openpyxl


In [7]:
import os
import numpy as np
from tqdm.auto import tqdm
from glob import glob
from transformers import AutoFeatureExtractor, pipeline
import pandas as pd
import librosa
import IPython
from datasets import load_metric
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
import torch
import gc
import wave
from scipy.io import wavfile
import scipy.signal as sps
import pyctcdecode

tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")



In [8]:
# CHANGE ACCORDINGLY
BATCH_SIZE = 16
TEST_DIRECTORY1 = '/kaggle/input/narail-1'

path1 = glob(os.path.join(TEST_DIRECTORY1,'*.wav'))
paths = path1
#print(paths[:2])

In [9]:
class CFG:
    my_model_name = '../input/yellowking-dlsprint-model/YellowKing_model'
    processor_name = '../input/yellowking-dlsprint-model/YellowKing_processor'


from transformers import Wav2Vec2ProcessorWithLM

processor = Wav2Vec2ProcessorWithLM.from_pretrained(CFG.processor_name)

In [10]:
my_asrLM = pipeline("automatic-speech-recognition", model=CFG.my_model_name ,feature_extractor =processor.feature_extractor, tokenizer= processor.tokenizer,decoder=processor.decoder ,device=0)


speech, sr = librosa.load('/kaggle/input/bengaliai-speech/test_mp3s/0f3dac00655e.mp3', sr=processor.feature_extractor.sampling_rate)

my_asrLM([speech]*2, chunk_length_s=112, stride_length_s=None)

my_asrLM



class AudioDataset(Dataset):
    def __init__(self, paths):
        self.paths = paths
    def __len__(self):
        return len(self.paths)
    def __getitem__(self,idx):
        speech, sr = librosa.load(self.paths[idx], sr=processor.feature_extractor.sampling_rate) 
#         print(speech.shape)
        return speech


dataset = AudioDataset(paths)
dataset[0]
device = 'cuda:0'

def collate_fn_padd(batch):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.
    '''
    ## get sequence lengths
    lengths = torch.tensor([ t.shape[0] for t in batch ])
    ## padd
    batch = [ torch.Tensor(t) for t in batch ]
    batch = torch.nn.utils.rnn.pad_sequence(batch)
    ## compute mask
    mask = (batch != 0)
    return batch, lengths, mask


dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=8, collate_fn=collate_fn_padd)



preds_all = []
for batch, lengths, mask in dataloader:
    preds = my_asrLM(list(batch.numpy().transpose()))
    preds_all+=preds


    from bnunicodenormalizer import Normalizer 


bnorm = Normalizer()
def normalize(sen):
    _words = [bnorm(word)['normalized']  for word in sen.split()]
    return " ".join([word for word in _words if word is not None])

def dari(sentence):
    try:
        if sentence[-1]!="।":
            sentence+="।"
    except:
        print(sentence)
    return sentence




df1= pd.DataFrame(
    {
        "External_ID":[p.split(os.sep)[-1].replace('.wav','') for p in paths],
        "content":[p['text']for p in preds_all]
    }
)
df1.content= df1.content.apply(lambda x:normalize(x))
df1.content= df1.content.apply(lambda x:dari(x))

In [11]:
df1

,External_ID,content
0,rec_19_audio_24,সে দিন নেচে নেভাল জ্যোতিদাদা সামির ছোট ডান সাথ...
1,rec_05_audio_0,বন্দনা জন দেবিকাকে মাসে আসে নিমসার আমরা তোমাদে...
2,rec_05_audio_18,জবাবদিহি কার কে গান ছবি দেখতে যেতে দেয়নি।
3,rec_08_audio_32,মেলাদিন কথাটথা করে না ইন্ডিয়ায় গেলনা ইন্ড্রাস্...
4,rec_18_audio_10,তার দিবারাত্রির।
...,...,...
995,rec_17_audio_21,তারপর মধ্যে রাইলি রাখতে বিজয় রাখতে তারপরে পানি...
996,rec_09_audio_19,কাতারের বর্তমান বিরত্তের পরে যা পারসেলকে গেছে ...
997,rec_04_audio_6,এবার আমার দিতে গল্লিতে স্বাস্থ এটি দূরের যে ছো...
998,rec_16_audio_7,ভালো ছিল লেখন ভাল হয়েছে করিনা অসুস্থ ছিল জর্জ ...


In [12]:
df1.to_csv("submission1.csv", index=False)

In [13]:
print('done')

done


In [ ]:
===========================================================================================================

In [14]:
# CHANGE ACCORDINGLY
BATCH_SIZE = 16
TEST_DIRECTORY2 = '/kaggle/input/narail-2'
path2 = glob(os.path.join(TEST_DIRECTORY2,'*.wav'))
paths = path2

In [15]:
my_asrLM = pipeline("automatic-speech-recognition", model=CFG.my_model_name ,feature_extractor =processor.feature_extractor, tokenizer= processor.tokenizer,decoder=processor.decoder ,device=0)


#speech, sr = librosa.load('/kaggle/input/bengaliai-speech/test_mp3s/0f3dac00655e.mp3', sr=processor.feature_extractor.sampling_rate)

my_asrLM([speech]*2, chunk_length_s=112, stride_length_s=None)

my_asrLM



class AudioDataset(Dataset):
    def __init__(self, paths):
        self.paths = paths
    def __len__(self):
        return len(self.paths)
    def __getitem__(self,idx):
        speech, sr = librosa.load(self.paths[idx], sr=processor.feature_extractor.sampling_rate) 
#         print(speech.shape)
        return speech


dataset = AudioDataset(paths)
#dataset[0]
device = 'cuda:0'

def collate_fn_padd(batch):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.
    '''
    ## get sequence lengths
    lengths = torch.tensor([ t.shape[0] for t in batch ])
    ## padd
    batch = [ torch.Tensor(t) for t in batch ]
    batch = torch.nn.utils.rnn.pad_sequence(batch)
    ## compute mask
    mask = (batch != 0)
    return batch, lengths, mask


dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=8, collate_fn=collate_fn_padd)



preds_all = []
for batch, lengths, mask in dataloader:
    preds = my_asrLM(list(batch.numpy().transpose()))
    preds_all+=preds


    from bnunicodenormalizer import Normalizer 


bnorm = Normalizer()
def normalize(sen):
    _words = [bnorm(word)['normalized']  for word in sen.split()]
    return " ".join([word for word in _words if word is not None])

def dari(sentence):
    try:
        if sentence[-1]!="।":
            sentence+="।"
    except:
        print(sentence)
    return sentence




df2= pd.DataFrame(
    {
        "External_ID":[p.split(os.sep)[-1].replace('.wav','') for p in paths],
        "content":[p['text']for p in preds_all]
    }
)

df2.content= df2.content.apply(lambda x:normalize(x))
df2.content= df2.content.apply(lambda x:dari(x))

In [16]:
df2

,External_ID,content
0,rec_32_audio_29,জমিনে কি জামিল মেলা করবে আমার তাই করছে করবার ট...
1,rec_26_audio_43,ইশামনী করিনায় জাতীয় জাকাতের কাতার তিনিমুসলিম র...
2,rec_23_audio_76,তা কনটিনিউটি ধরা নিয়ে গেছে শিটের সব এসব খুবই ন...
3,rec_35_audio_36,আমি টেবি তারিক তাবারে কালকে যে সন্তষ্টদেশ খাওয়...
4,rec_27_audio_3,পরকে প্রায় মেসির কাছে হয়ে স্ট্যান্ডের কাছ পারছ...
...,...,...
875,rec_29_audio_10,তাকে বারেককে যাবে জামাইরে নিয়ে করতে পরিঘটনার ব...
876,rec_23_audio_40,এসব করে করা কনেকে বারিসান কথার বন্ধুকে বাড়িতে ...
877,rec_32_audio_49,তা আমরা খাইনে তারা খালে খালেদার খাই নে মা তাকা...
878,rec_26_audio_51,এর দাদাবাড়িতে কাব্যে একাত্তর পঞ্চাননের কারখানা...


In [23]:
df2.to_csv("submission2.csv", index=False)

In [24]:
print('done')

done


In [ ]:
=================================================================================================================

In [28]:
# import pandas as pd
# path1 = "submission1.csv"
# path2 = "submission2.csv"
# path3 = "submission3.csv"


# df1 = pd.read_csv(path1)
# df2 = pd.read_csv(path2)
# df3 = pd.read_csv(path3)

values1 = df1[['External_ID','content']]
values2 = df2[['External_ID','content']]


dataframes = [values1, values2]

join = pd.concat(dataframes)
join

,External_ID,content
0,rec_19_audio_24,সে দিন নেচে নেভাল জ্যোতিদাদা সামির ছোট ডান সাথ...
1,rec_05_audio_0,বন্দনা জন দেবিকাকে মাসে আসে নিমসার আমরা তোমাদে...
2,rec_05_audio_18,জবাবদিহি কার কে গান ছবি দেখতে যেতে দেয়নি।
3,rec_08_audio_32,মেলাদিন কথাটথা করে না ইন্ডিয়ায় গেলনা ইন্ড্রাস্...
4,rec_18_audio_10,তার দিবারাত্রির।
...,...,...
875,rec_29_audio_10,তাকে বারেককে যাবে জামাইরে নিয়ে করতে পরিঘটনার ব...
876,rec_23_audio_40,এসব করে করা কনেকে বারিসান কথার বন্ধুকে বাড়িতে ...
877,rec_32_audio_49,তা আমরা খাইনে তারা খালে খালেদার খাই নে মা তাকা...
878,rec_26_audio_51,এর দাদাবাড়িতে কাব্যে একাত্তর পঞ্চাননের কারখানা...


In [30]:
join.to_csv("2.2: narail-yellowking-inference.csv")

In [22]:
print('done')

done
